# Web Scraping Teil 2

Willkommen zurück! 

Wir machen direkt weiter mit ein paar Informationen zu verantwortungsvollem Scrapen. Darauf folgt der bereits angeteaserte Anwendungsfall: Wir scrapen den Faust I. Das Notebook endet mit einem Exkurs zu der HTML-verwandten Auszeichnungssprache *XML*, die besonders in den Digital Humanities gerne verwendet wird, um Daten zu speichern und zu teilen.

Zunächst führen wir diese Zelle aus:

In [1]:
import requests
from bs4 import BeautifulSoup
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'}

## Recht und Ethik

Rechtlich gesehen ist Web Scraping zu Forschungszwecken vollkommen legal ([EU-Gesetz dazu](https://eur-lex.europa.eu/legal-content/DE/TXT/PDF/?uri=CELEX:32019L0790)). Wir müssen dabei lediglich beachten, dass wir nur öffentlich zugängliche Daten scrapen – die hier erlernten Methoden gestatten uns aber ohnehin nur das. Wenn wir unsere gescrapten Daten weiterveröffentlichen, etwa in einem Paper, dann müssen wir außerdem bedenken, personenbezogene Daten zu anonymisieren. 

Bei der Ethik wiederum geht es nicht darum, was legal ist, sondern darum, was nach gesundem Menschenverstand in Ordnung ist. Webseiten werden ja auf Servern betrieben, die für eine bestimmte Auslastung (also eine bestimmte Anzahl an Anfragen pro Zeiteinheit) ausgelegt sind. Wenn Server überlastet werden, haben nicht nur wir ein Problem, sondern auch andere Leute, die die Webseite aufrufen möchten. Verantwortungsvolles Web Scraping bedeutet folglich, nur so viele Anfragen an den Server zu schicken, wie nötig. 

Die Aufteilung von Abruf- und Extraktionsschritt in verschiedene Zellen eines Jupyter Notebooks ist eine Maßnahme, die verhindert, dass wir den Server selbst dann belasten, wenn wir den Abrufschritt längst absolviert haben. Wenn wir im Abrufschritt an sich eine hohe Anzahl an Quelltexten herunterladen wollen, können wir außerdem einen Timer einbauen, der zwischen den einzelnen Anfragen eine kurze Pause einlegt. Dazu benutzen wir das Modul `time`, das zur Grundausstattung von Python gehört und folglich nicht erst installiert werden muss:

In [2]:
import time

links = ["https://de.wikipedia.org/wiki/Althochdeutsche_Sprache",
         "https://de.wikipedia.org/wiki/Mittelhochdeutsche_Sprache",
         "https://de.wikipedia.org/wiki/Frühneuhochdeutsche_Sprache"]

source_codes = []  

for link in links:
    
    response = requests.get(link, timeout=5, headers=headers)
    source_codes.append(response.text)
    
    print(f"Fertig abgerufen: {link}. Nun wird kurz pausiert.") #Zur besseren Nachvollziehbarkeit des Prozesses
    
    #Der Funktion 'sleep' übergeben wir, wie viele Sekunden pausiert werden soll
    time.sleep(5)

Fertig abgerufen: https://de.wikipedia.org/wiki/Althochdeutsche_Sprache. Nun wird kurz pausiert.


Fertig abgerufen: https://de.wikipedia.org/wiki/Mittelhochdeutsche_Sprache. Nun wird kurz pausiert.


Fertig abgerufen: https://de.wikipedia.org/wiki/Frühneuhochdeutsche_Sprache. Nun wird kurz pausiert.


Manchmal übertreiben wir es mit den Anfragen aus Versehen trotzdem und wir werden vom Server blockiert. In anderen Fällen wiederum werden wir selbst bei verantwortungsvollem Web Scraping blockiert, da der Server Anti-Scraping-Mechanismen implementiert hat und unsere Anfrage als automatisiert erkannt wurde. Eine Blockierung kann sich verschiedentlich äußern, u.&nbsp;a. in den HTTP-Statuscodes 403 ("Forbidden") bzw. 429 ("Too many requests").

Beim Abrufen einer hohen Anzahl an Quelltexten lohnt es sich also zusätzlich zur Pausierung einen Mechanismus zur Kontrolle der Statuscodes einzubauen:

In [3]:
links = ["https://de.wikipedia.org/wiki/Althochdeutsche_Sprache",
         "https://de.wikipedia.org/wiki/Mittelhochdeutsche_Sprache",
         "https://de.wikipedia.org/Dieser_Link_funktioniert_nicht",
         "https://de.wikipedia.org/wiki/Frühneuhochdeutsche_Sprache"]

source_codes = []  

for link in links:
    
    response = requests.get(link, timeout=5, headers=headers)
    
    #Kontrolle des Statuscodes mithilfe eines in 'requests' eingebauten Mechanismus
    if not response.status_code == requests.codes.ok:
        print("Statuscode nicht ok!")
        break
        
    source_codes.append(response.text)

    print(f"Fertig abgerufen: {link}. Nun wird kurz pausiert.") #Zur besseren Nachvollziehbarkeit des Prozesses
    
    #Der Funktion 'sleep' übergeben wir, wie viele Sekunden pausiert werden soll (hier fünf Sekunden)
    time.sleep(5)

Fertig abgerufen: https://de.wikipedia.org/wiki/Althochdeutsche_Sprache. Nun wird kurz pausiert.


Fertig abgerufen: https://de.wikipedia.org/wiki/Mittelhochdeutsche_Sprache. Nun wird kurz pausiert.


Statuscode nicht ok!


Die Abrufschleife bricht ab, sobald der Statuscode auf einen Fehler hindeutet. Hier trifft das beim dritten Link zu. Nicht, weil wir blockiert worden wäre, sondern weil der Link schlicht nicht existiert (Statuscode 404). 

Die bereits erfolgreich abgerufenen Quelltexte bleiben uns dank diesem Code aber erhalten:

In [4]:
len(source_codes)

2

Für den Fall, dass Du mal blockiert wirst, kannst Du versuchen, andere oder weitere Parameter in `headers` zu spezifizieren oder ggf. sogar zwischen unterschiedlichen `headers` zu rotieren. Dadurch sieht es so aus, als kämen die Anfragen von unterschiedlichen Browsern. Eine weitere Möglichkeit besteht darin, über einen VPN oder sog. *Proxies* Deine IP-Adresse zu ändern bzw. ebenfalls zwischen verschiedenen IP-Adressen zu rotieren. Informier Dich bei Bedarf im Internet zu diesen Methoden sowie in der [Dokumentation](https://requests.readthedocs.io/en/latest/user/advanced/#proxies) von `requests`.

Wenden wir uns dem Anwendungsfall zu:

***

## 🔧 Anwendungsfall: Den Faust I scrapen 🎭

Nun wollen wir den kompletten *Faust I* von [Projekt Gutenberg](https://www.projekt-gutenberg.org/goethe/faust1/chap002.html) scrapen und in einer Textdatei in folgendem Format abspeichern (der Auszug entstammt der Szene [*Nacht*](https://www.projekt-gutenberg.org/goethe/faust1/chap004.html)):

    Geist

        Wer ruft mir?

    Faust

        Schreckliches Gesicht!

    Geist

        Du hast mich mächtig angezogen,
        An meiner Sphäre lang gesogen,
        Und nun –

    Faust

        Weh! ich ertrag dich nicht!
        
Du kannst nun entweder den Anwendungsfall ohne weitere Anleitung in Angriff nehmen. An der ein oder anderen Stelle wirst Du vermutlich nach Lösungen im Internet suchen müssen. Da dies außerhalb dieser Lernumgebung ohnehin häufig vorkommen wird, stellt dies eine gute Übungsgelegenheit dafür dar. Andererseits steht Dir im Anschluss an die beiden Code-Zellen eine Schritt-für-Schritt-Anleitung zur Verfügung, die Dich durch den Anwendungsfall lotst. 

In jedem Fall: viel Erfolg! 🙌

In [5]:
#In diese Zelle kannst Du den Code für den Abrufschritt schreiben.
#Führe diese Zelle nur so oft wie nötig aus, um den Server nicht unnötig zu belasten!





















In [6]:
#In diese Zelle kannst Du den Code für den Extraktionsschritt schreiben.





















*** 

**Schritt-für-Schritt-Anleitung**

<u>Abruf</u>

1. Schau Dir die [erste Seite](https://www.projekt-gutenberg.org/goethe/faust1/chap002.html) des Werks auf www.projekt-gutenberg.org im Browser an und untersuch den ihr zugrundeliegenden Quelltext. Find so heraus, wie Du an die Links zu den folgenden Seiten kommst, damit Du auch diese scrapen kannst.

2. Wie Du im Quelltext der ersten Seite erkennen kannst, verbirgt sich hinter der Schaltfläche "Weiter" nur jeweils die Linkendung, die um einen Stammlink ergänzt werden muss. 
    
    Definier den Stammlink in `base_link` sowie die Linkendung für die erste Seite in `link_ending`. Konkatenier die beiden strings zu `link` und lass Dir diesen ausgeben. Er sollte Dich zur ersten Seite des Werks führen.

In [7]:
base_link = ""
link_ending = ""



3. Überleg Dir nun, mithilfe welcher Kontrollstruktur Du ausgehend von der ersten Seite nacheinander alle Seiten scrapen kannst, bis es keine weitere Seite mehr gibt. Welche weitere Kontrollstruktur kannst Du verwenden, um das Scraping in diesem Fall zu beenden? 

4. Um von einer Seite zur nächsten zu gelangen, müssen wir ja jeweils den Link darauf aus der aktuellen Seite extrahieren. Definier dafür einen regulären Ausdruck, der den Link hinter der Schaltfläche "Weiter" matcht. Falls Du noch nicht mit regulären Ausdrücken vertraut bist, dann kopier den Code für diesen Schritt aus der Lösung.

    <details>
      <summary>💡 Tipp </summary>
      <br>Lass Deinen regulären Ausdruck ruhig mehr als nur den eigentlichen Link matchen, also auch Zeichen davor bzw. danach im Quelltext. So stellst Du einfach sicher, dass nur der gewünschte Link gematcht wird. Bau aber mithilfe von runden Klammern eine Gruppe ein, die nur den Link umfasst (vgl. Notebook "Reguläre Ausdrücke"). So kannst Du anschließend einfach auf den relevanten Teil des matches zugreifen.
    </details>
    <br>

In [8]:
regex = r""

5. Setz nun Deine Erkenntnisse aus Schritt 3 in Code um: 
    
    A. Schreib eine Schleife (erste Kontrollstruktur), die wiederholt wird, bis sie auf ein `break`-Statement trifft. 
    
    B. Verwend das `request`-Modul, um die Seite zum aktuellen `link` abzurufen (`link` entspricht ja zumindest am Anfang der ersten Seite, s.&nbsp;o.). Speichere das Response-Objekt in `current_page`.
    
    C. Da `requests` automatisch von einem falschen Encoding ausgeht (weswegen etwa Umlaute falsch dekodiert würden), müssen wir das Encoding korrigieren: `current_page.encoding = "UTF-8"`.
    
    D. Häng den eigentlichen Quelltext in `current_page` einer zuvor definierten Liste `all_pages` an, die sämtliche Quelltexte umfassen soll.
    
    E. Such den Quelltext nach `regex` ab. Verwend dazu die Funktion `search` des Moduls `re` (schau auch hier in der Lösung nach, wenn Du mit regulären Ausdrücken noch nicht vertraut bist).
    <details>
      <summary>💡 Tipp </summary>
      <br>Vergiss nicht, <code>re</code> erst zu importieren.<br>
    </details>
    <br>
    
    F. Überprüf nun mithilfe der zweiten Kontrollstruktur, ob ein match vorliegt. Wenn ja, überschreib `link`, indem Du den match, also die neue Linkendung, an `base_link` anhängst. Nun kann die nächste Seite gescrapt werden. Liegt kein match (mehr) vor, soll die Schleife abgebrochen werden.    
    <details>
      <summary>💡 Tipp </summary>
      <br>Die zum matchen benutzten Funktion <code>search</code> gibt ja ein sog. match-Objekt zurück, bestehend aus dem ersten gefundenen match und dessen Position im abgesuchten string. Nun sind wir nicht am match als Ganzes interessiert, sondern nur an der Gruppe mit der Linkendung. Wend die Methode <code>group</code> auf das match-Objekt an und übergib ihr in Klammern eine Eins, um direkt auf die erste (und einzige) Gruppe zuzugreifen (vgl. Notebook "Reguläre Ausdrücke").
    <br>
    </details>
    <br>

<u>Extraktion</u>

6. Nun haben wir alle Quelltexte gescrapt und es geht ans Extrahieren der relevanten Daten. Wir schreiben zunächst Code, um alle Strophen sowie die Figuren aus einem *einzelnen* Quelltext zu extrahieren. Anschließend bauen wir diesen Code in eine Schleife, der über *sämtliche* Quelltexte iteriert und nach und nach das gesamte Werk extrahiert.

    Analysier als Erstes die Quelltexte eingehend, entweder im Browser, in Sublime Text oder indem Du sie Dir mithilfe von `prettify` von BeautifulSoup ausgeben lässt. Welche Elemente mit welchen Tags und ggf. welchen Attributen beinhalten die Strophen und Figuren?

7. Verwend BeautifulSoup, um den ersten Quelltext zu parsen. Schaff ein Objekt `body`, das nur noch das `<body>`-Element beinhaltet.

8. Iterier mithilfe von `find_all` über alle Elemente desjenigen Tags, das sowohl Strophen als auch Figuren umfasst. 

    Überprüf für jedes Element erstens, ob sich darin untergeordnet dasjenige Element befindet, das die Figuren enthält. Wenn ja, extrahier es und weis es `speaker` zu. Bereinige `speaker` von überflüssigen Zeichen und lass es Dir ausgeben.

    <details>
      <summary>💡 Tipp</summary>
      <br>Es ist wichtig, dass Du zunächst mithilfe einer <code>if</code>-Bedingung überprüfst, ob ein solches Element gefunden werden kann, bevor Du auf dessen Inhalt zugreifst. Existiert das Element nämlich nicht, würde der Zugriff eine Fehlermeldung auslösen.
    </details>
    <br>
    
    Überprüf zweitens, ob das Element andernfalls über dasjenige Attribut verfügt, das sämtliche Elemente, die Strophen beinhalten, miteinander teilen. Wenn ja, extrahier es, unterteil es in Verse und weis diese `lines` zu. Lass Dir `lines` schön formatiert ausgeben.
    
    <details>
      <summary>💡 Tipp</summary>
      <br>Verwend die <code>get</code>-Methode, um zu überprüfen, ob das gewünschte Attribut dem gesuchten Wert entspricht. Find außerdem heraus, ob der Wert des Attributs wirklich als string zurückgegeben wird. 
    </details>
    <br>
    
    ⚠️ Achtung: Manche Strophen beinhalten auch Regieanweisungen in einem untergeordneten `<span>`-Element. Bau folgenden Code an der richtigen Stelle ein, um diese `<span>`-Elemente aus den Strophen zu entfernen:
    
    `if line.span:`
     <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`line.span.decompose()`
    
    Wir überprüfen damit erst, ob die entsprechende Strophe ein `<span>`-Element enthält und wenn ja, entfernen wir es aus ihr mithilfe der `decompose`-Methode.

9. Modifizier den Code aus Schritt 8 derart, dass Du die relevanten Daten nicht nur aus *einem* Quelltext extrahierst, sondern aus *allen* auf `all_pages`. Pass ihn außerdem so an, dass Dir Figuren und Strophen nicht ausgegeben werden, sondern dass diese in eine externe Datei geschrieben werden. 

    <details>
          <summary>💡 Tipp</summary>
          <br>Beachte, dass die <code>print</code>-Funktion automatisch einen Zeilenumbruch im Anschluss an das Ausgegebene anfügt. Beim Schreiben in externe Dateien musst Du diesen Zeilenumbruch manuell hinzufügen. 
        </details>
        <br>

Tolle Arbeit! 🎉

***

Zu guter Letzt werfen wir noch einen Blick auf die HTML-verwandte Auszeichnungssprache XML. Während HTML primär dazu verwendet wird, Texte und Daten *zwecks der Darstellung* zu strukturieren (bzw. *auszuzeichnen*), wird XML vorwiegend dazu benutzt, Texte und Daten *zwecks der Speicherung* zu gliedern. 

## Exkurs: XML

Ein XML-Dokument (kurz für *E<u>x</u>tensible <u>M</u>arkup <u>L</u>anguage*) ist im Wesentlich gleich strukturiert wie ein in HTML verfasstes: Verschiedene **Elemente** sind in einem Stammbaum **hierarchisch** angeordnet. Die Elemente sind wiederum durch **Tags** sowie optional **Attribute** charakterisiert. 

Folgende Zelle enthält ein Beispiel für XML-Code. Die erste Zeile deklariert den Dokumenttyp – das kennen wir auch von HTML-Code. Es folgt ein Verzeichnis (`<verzeichnis>`), dessen Inhalt neben einem `<titel>`-Element zwei `<eintrag>`-Elemente umfasst. Jedes `<eintrag>`-Element verfügt über ein `id`-Attribut sowie drei Kinder mit Textinhalt, nämlich `<stichwort>`, `<abkuerzung>` und `<zweck>`. Die Einrückungen dienen auch hier nur der Lesbarkeit.

An diesem Beispiel lässt sich bereits der wichtigste Unterschied zwischen XML und HTML erkennen: Die Namen der Tags sind nicht vorgegeben, sondern können frei gewählt werden (u.&nbsp;a. daher kommt die Bezeichnung *extensible*, also "erweiterbar"). 

Arbeiten wir ganz praktisch mit einem XML-Dokument! Konkret schauen wir uns ein sog. *Plenarprotokoll* des Deutschen Bundestags an, und zwar das vom 10. April 2024. Darin befinden sich sämtliche Äußerungen der Abgeordneten, die an diesem Tag im Parlament gemacht wurden, stenografisch protokolliert. Ein solches Protokoll im XML-Format können wir für sämtliche Sitzungen des Bundestags [hier](https://www.bundestag.de/services/opendata) herunterladen. 

### XML parsen

Das ausgewählte Protokoll befindet sich bereits in "3_Dateien/XML". Wie bis anhin gilt: Öffne die Datei am besten parallel zu diesem Notebook in Sublime Text. So fällt es Dir leichter, die folgenden Überlegungen im Code nachzuvollziehen.

<details>
<summary>💡 Tipp</summary>
<br>Benutz das im GIF unten gezeigte, praktische Feature, um Elemente ein- und auszuklappen. Dadurch siehst Du schnell, in welcher Beziehung die einzelnen Elemente zueinander stehen. Dieses Feature funktioniert übrigens auch bei HTML-Code.<br><br>
    <img src="../3_Dateien/Grafiken_und_Videos/einklappen.gif" width="600">
</details>
<br>

Um die Datei hier einzulesen, importieren wir das Modul `xml` (bzw. einen spezifischen Teil davon). `xml` gehört bei Python zur Grundausstattung.

In [9]:
import xml.etree.ElementTree as ET #Wir weisen dem Modul(teil) den Kurznamen 'ET' zu, was der Konvention entspricht.

⚠️ Achtung: Viele der folgenden Techniken des Moduls `xml` ähneln jenen von `BeautifulSoup` für HTML-Code. Bei den meisten gibt es aber kleine, teils sehr bedeutsame Unterschiede in der Implementierung. Ein Beispiel: Die Methoden `find` und `find_all` (`BeautifulSoup`) bzw. `findall` (`xml`) dienen bei beiden Modulen der Suche nach dem ersten resp. allen Elementen, die bestimmte Kriterien erfüllen. Während jedoch `BeautifulSoup` sog. *rekursiv*  sucht, also in sämtlichen untergeordneten Elementen (und den Elementen, die diesen untergeordnet sind, etc.), beschränkt sich `xml` auf das Element, auf das wir die Methode anwenden. Für die rekursive Suche hält das Modul stattdessen die Methode `iter` bereit (s.&nbsp;u.). Behalt die Tatsache, dass es sich um zwei verschiedene Module, denen unterschiedlicher Code zugrunde liegt, einfach im Hinterkopf. Wo relevant, wird in den folgenden Code-Zellen auf Pendants bei `BeautifulSoup` verwiesen.

Um nun das Protokoll zu laden und gleich zu parsen, verwenden wir bei `xml` die `parse`-Funktion. Anschließend müssen wir noch das oberste Element, also den Stamm bzw. die Wurzel, über die Methode `getroot` extrahieren:

In [10]:
tree = ET.parse("../3_Dateien/XML/plenarprotokoll.xml")
root = tree.getroot() 

Was könnte sich in `root` nun befinden? Schau dazu im Dokument bei Sublime Text nach.

Genau, das gesamte oberste Element unseres XML-Dokuments, also dasjenige mit dem Tag `<dbtplenarprotokoll>`. Das können wir folgendemaßen überprüfen:

In [11]:
root.tag #'tag' entspricht bei 'BeautifulSoup' dem 'name'-Attribut.

'dbtplenarprotokoll'

Wie Du im Dokument sehen kannst, verfügt dieses Element über diverse Attribute. Diese können wir uns wie folgt ausgeben lassen:

In [12]:
root.attrib #'attrib' entspricht bei 'BeautifulSoup' dem 'attrs'-Attribut.

{'vertrieb': 'Bundesanzeiger Verlag GmbH, Postfach 1 0 05 34, 50445 Köln, Telefon (02 21) 97 66 83 40, Fax (02 21) 97 66 83 44, www.bundesanzeiger-verlag.de',
 'herstellung': 'H. Heenemann GmbH  Co. KG, Buch- und Offsetdruckerei, Bessemerstraße 83–91, 12103 Berlin, www.heenemann-druck.de',
 'sitzung-ort': 'Berlin',
 'herausgeber': 'Deutscher Bundestag',
 'issn': '0722-7980',
 'wahlperiode': '20',
 'sitzung-nr': '162',
 'sitzung-datum': '10.04.2024',
 'sitzung-start-uhrzeit': '13:00',
 'sitzung-ende-uhrzeit': '19:40',
 'sitzung-naechste-datum': '11.04.2024',
 'start-seitennr': '20745'}

Einzelne Werte zu bestimmten Attributen (bzw. Schlüsseln) können wir so extrahieren:

In [13]:
root.get("sitzung-datum") #Gleichnamige Methode wie bei 'BeautifulSoup'

'10.04.2024'

Um sämtliche Elemente mit einem bestimmten Tag **rekursiv** zu extrahieren, benutzen wir bei `xml` wie erwähnt die `iter`-Methode, am besten direkt in eine Schleife eingebaut.

In [14]:
for item in root.iter("tagesordnungspunkt"):
    print(item.tag, item.attrib)

tagesordnungspunkt {'top-id': 'Tagesordnungspunkt 1'}
tagesordnungspunkt {'top-id': 'Tagesordnungspunkt 2'}
tagesordnungspunkt {'top-id': 'Zusatzpunkt 1'}
tagesordnungspunkt {'top-id': 'Tagesordnungspunkt 3'}
tagesordnungspunkt {'top-id': 'Zusatzpunkt 2'}
tagesordnungspunkt {'top-id': 'Zusatzpunkt 3'}
tagesordnungspunkt {'top-id': 'Tagesordnungspunkt 5'}


Am 10. April 2024 wurden also insgesamt sieben Tagesordnungspunkte abgehandelt. 

Angenommen wir sind nur an den "echten" Tagesordnungspunkten und nicht an den "Zusatzpunkten" interessiert, so können wir zusätzlich zum Tag nach dem Attribut `top-id` filtern. `iter` – ebenso wie die nicht-rekursiven Methoden `find` und `findall` – akzeptiert allerdings maximal ein Tag als Suchkriterium und keine Liste oder Kombination von Tags und Attributen. Stattdessen müssen wir zu einer `if`-Bedingung greifen:

In [15]:
for item in root.iter("tagesordnungspunkt"):
    #Überprüfung, ob der Wert des Attributs bzw. des Schlüssels "top-id", der ja ein string ist, mit "T" beginnt
    if item.get("top-id").startswith("T"):
        print(item.tag, item.attrib)

tagesordnungspunkt {'top-id': 'Tagesordnungspunkt 1'}
tagesordnungspunkt {'top-id': 'Tagesordnungspunkt 2'}
tagesordnungspunkt {'top-id': 'Tagesordnungspunkt 3'}
tagesordnungspunkt {'top-id': 'Tagesordnungspunkt 5'}


Klappt hervorragend!

Nun wollen wir uns auf die Äußerungen der Abgeordneten im Protokoll konzentrieren. Wie ein Blick in das XML-Dokument verrät, befinden sich diese in Elementen mit dem Tag `<rede>` bzw. diesem untergeordnet in `<p>`-Elementen. In der folgenden Zelle verwenden wir wieder `iter` – zweimal hintereinander geschaltet – und extrahieren den in den `<p>`-Elementen befindlichen Textinhalt über das `text`-Attribut. Die `if.paragraph.text`-Bedingung ist nötig, da gewisse `<p>`-Elemente leer sind und `None` zurückgeben würden.

In [16]:
for speech in root.iter("rede"):
    for paragraph in speech.iter("p"):
        if paragraph.text:
            print(paragraph.text.strip(), end=" ") #'text' heißt gleich bei 'BeautifulSoup'
    print("\n") #Einfügen eines Zeilenumbruchs nach jeder Rede

Einen schönen guten Tag, Frau Präsidentin! Liebe Kolleginnen und Kollegen! Als ich im Oktober zum letzten Mal hier in der Regierungsbefragung stand, war der furchtbare Angriff der Hamas auf Israel erst wenige Tage her. Seitdem, seit sechs Monaten, treiben uns Sorgen um die Geiseln und das unsägliche Leid der Menschen in Gaza um. Zwischendurch haben wir in all den Verhandlungen immer wieder nicht nur gehofft, dass wir diesen Krieg beenden können, sondern auch auf eine Chance auf einen politischen Horizont, auf ein Ende des Konflikts gehofft; aber leider sind wir dort nicht. Zugleich haben wir, habe ich persönlich erlebt, was Außenpolitik in diesen Zeiten bewirken kann, vor allen Dingen Pendeldiplomatie und Außenpolitik, die vielfach hinter verschlossenen Türen stattfindet. Sie ist aus meiner Sicht in diesen so herausfordernden Zeiten wichtiger denn je, weil unser beharrliches Ringen und Verhandeln über Grenzübergänge, über jeden einzelnen Lkw hinter verschlossenen Türen einen Unterschie

Das scheint ganz gut zu klappen. 

Überprüfen wir aber zur Sicherheit, ob wir auch wirklich nur das Gewünschte extrahiert haben, also die Äußerungen der Abgeordneten (bzw. die *True Positives*, vgl. "Die Wahrheitsmatrix" in Notebook "Input und Output Teil 1"). Beim Lesen der ersten Rede von Annalena Baerbock irritiert der Schluss (alles nach "Vielen Dank"). Dieser wurde, wie ein Blick in das XML-Dokument zeigt, denn auch gar nicht von der Rednerin selbst gesprochen, sondern von der Parlamentspräsidentin Bärbel Bas. Einziger Hinweis ist ein dem fraglichen `<p>`-Element vorgeschobenes `<name>`-Element. Tatsächlich finden sich solche Äußerungen der Parlamentspräsidentin (und später im Protokoll ihres Vize-Präsidenten Wolfgang Kubicki) am Ende der meisten `<rede>`-Elemente – jeweils mit vorgeschobenem `<name>`-Element. Diese gehören natürlich nicht zur Äußerung des/der Abgeordneten der anderen `<p>`-Elemente unter `<rede>` und sollten somit nicht extrahiert werden.

Idealerweise ließen sich die `<p>`-Elemente, die wirklich zur Äußerung des/der Abgeordneten gehören von jenen, die von jemand anderem geäußert werden, über unterschiedliche Tags voneinander abgrenzen. Bei Zwischenrufen ist dies der Fall, sind diese doch in `<kommentar>`-Elementen statt in `<p>`-Elementen gespeichert. Alternativ hätten `<p>`-Elemente, die nicht von dem/der betreffenden Abgeordneten stammen, über Attribute charakterisiert werden können. Doch auch das ist nicht der Fall, wie ein Abgleich mit dem Dokument zeigt. 

Wenn wir nicht über Tags bzw. Attribute gehen können, bleibt der Weg über die Reihenfolge der Elemente – wie erwähnt befinden sich die eingeschobenen Äußerungen der Parlamentspräsidentin meist am Ende eines `<rede>`-Elements. Wir können den Code also derart anpassen, dass wir statt bloß über `<p>`-Elemente über alle Kinder jedes `<rede>`-Elements iterieren. Dafür können wir `speech` wie ein gewöhnliches iterierbares Objekt behandeln, das uns nach und nach seine Kinder zurückgibt. Entscheidenderweise brechen wir die Schleife ab, sobald wir auf ein Kind mit `<name>`-Tag treffen:

In [17]:
for speech in root.iter("rede"):
    for element in speech:
        if element.tag == "name":
            break
        elif element.tag == "p" and element.text:
            print(element.text.strip(), end=" ") #Inkl. Bereinigung von leading/trailing whitespace
    print("\n") #Einfügen eines Zeilenumbruchs nach jeder Rede

Einen schönen guten Tag, Frau Präsidentin! Liebe Kolleginnen und Kollegen! Als ich im Oktober zum letzten Mal hier in der Regierungsbefragung stand, war der furchtbare Angriff der Hamas auf Israel erst wenige Tage her. Seitdem, seit sechs Monaten, treiben uns Sorgen um die Geiseln und das unsägliche Leid der Menschen in Gaza um. Zwischendurch haben wir in all den Verhandlungen immer wieder nicht nur gehofft, dass wir diesen Krieg beenden können, sondern auch auf eine Chance auf einen politischen Horizont, auf ein Ende des Konflikts gehofft; aber leider sind wir dort nicht. Zugleich haben wir, habe ich persönlich erlebt, was Außenpolitik in diesen Zeiten bewirken kann, vor allen Dingen Pendeldiplomatie und Außenpolitik, die vielfach hinter verschlossenen Türen stattfindet. Sie ist aus meiner Sicht in diesen so herausfordernden Zeiten wichtiger denn je, weil unser beharrliches Ringen und Verhandeln über Grenzübergänge, über jeden einzelnen Lkw hinter verschlossenen Türen einen Unterschie

Das klappt, wie wir etwa am Ende der ersten Rede erkennen können. 

Wie gewohnt müssen wir uns an dieser Stelle aber versichern, dass wir nicht "übers Ziel hinausgeschossen" sind und Äußerungen, die tatsächlich der/die jeweilige Abgeordnete gemacht hat, miteliminiert haben, weil ein `<name>`-Element noch vor dem Ende der Rede auftritt (etwa eine zwischenzeitliche Ordnungsbemerkung der Parlamentspräsidentin). Tatsächlich gibt es wenige solche Fälle, etwa in Zeile 2357.

Dieses Beispiel veranschaulicht zweierlei: Einerseits unterstreicht es, wie wichtig die vertiefte und repetitive Auseinandersetzung mit Daten ist. Andererseits zeigt es, dass eine einfache Programmierlösung manchmal schlicht nicht existiert. Sicherlich könnten wir weitere Bedingungen aufstellen, um die Extraktion zu perfektionieren. Je mehr es sich jedoch um Sonderfälle handelt, desto mehr sinkt der Nutzen einer maschinellen Herangehensweise. Abhängig vom Forschungsinteresse könnte man sich die verbleibenden *edge cases*  zur manuellen Entscheidung ausgeben lassen. Insbesondere bei quantitativen Vorhaben wäre aber auch ein gewisses Maß an Imperfektion der extrahierten Daten vertretbar.

Nun üben wir die Extraktion von Daten in XML-Dokumenten, wobei wir in Kauf nehmen, dass wir die Äußerungen von Abgeordneten nicht ganz fehlerfrei extrahieren können.

***

✏️ **Übung 1:** Pass obigen Extraktionscode so an, dass vor jeder Äußerung der Name des/der jeweiligen Abgeordneten ausgegeben wird. 

<details>
<summary>💡 Tipp</summary>
<br>Setz den Namen am besten aus Vor- und Nachnamen zusammen. Bedenk außerdem, dass nur eine Methode bei <code>xml</code> rekursiv sucht.<br><br>
</details>
<br>

In [18]:
#In diese Zelle kannst Du den Code zur Übung schreiben.




***

✏️ **Übung 2:** Neben den Reden bzw. Äußerungen der Abgeordneten, die gerade das Rederecht besitzen, enthält das Protokoll auch verbale Zwischenrufe sowie Anmerkungen über Beifall, Lachen etc. Diese Informationen sind in Elementen mit dem Tag `<kommentar>` enthalten. Extrahier sie für alle Reden, für die mindestens ein `<kommentar>`-Element protokolliert wurde. Lass sie Dir zusammen mit dem Namen des/der Abgeordneten ausgeben, der/die eigentlich gerade am sprechen ist.


In [19]:
#In diese Zelle kannst Du den Code zur Übung schreiben.




***

Sehr gut! 

Zum Abschluss lernen wir noch, wie wir selbst XML-Dokumente schreiben können. Wir exerzieren dies am Beispiel der in Übung 1 extrahierten Daten durch und speichern alle Äußerungen inklusive der Information, von wem sie stammen, in einer Datei im XML-Format.

### XML schreiben

Der erste Schritt beim Erstellen eines XML-Dokuments besteht darin, das oberste Element in der Hierarchie zu initialisieren und ihm einen Namen zuzuweisen. Wir nennen es schlicht `corpus` und weisen es einer gleichnamigen Variablen zu:

In [20]:
corpus = ET.Element("corpus")

Angenommen wir fertigen so ein Dokument für mehrere Sitzungen des Bundestags an, so macht es Sinn, Informationen etwa zum Datum der gegebenen Sitzung als Attribute des `corpus`-Elements zu speichern – wie das im originalen Plenarprotokoll auch getan wurde. Attribute *setzen* wir mit der `set`-Methode:

In [21]:
corpus.set("date", "10.04.2024") #Beachte, dass Schlüssel und Wert kommasepariert sind.

Die extrahierten Äußerungen bestehen ja typischerweise aus mehreren Paragraphen. Diese wollen wir nun pro Äußerung in je einem eigenen, `corpus` untergeordneten Element speichern. Das Schaffen dieser "Kinder" können wir direkt in die in Übung 1 verwendete Extraktionsschleife integrieren. Anstatt uns die extrahierten Daten per `print` auszugeben, speichern wir sie in einem `xml`-Element. 

Lies die beiden Kommentare im folgenden Code, um zu verstehen, wie Elemente in einem XML-Stammbaum angelegt werden:

In [22]:
for speech in root.iter("rede"):

    name = list(speech.iter("vorname"))[0].text
    surname = list(speech.iter("nachname"))[0].text
    
    """Für jede Äußerung schaffen wir ein Element namens 'statement' (zweites Argument der 'SubElement'-Funktion), 
    das dem 'corpus'-Element untergeordnet sein soll  (erstes Argument). 'statement' soll jeweils über ein Attribut
    namens 'speaker' verfügen, dessen Wert der zusammengesetzte Name des/der Abgeordneten ist (Achtung: das Schlüssel-
    Werte-Paar wird hier, anders als oben, wie bei einem dictionary geschaffen)."""
    statement = ET.SubElement(corpus, "statement", {"speaker": name + " " + surname})
    
    for element in speech:
        if element.tag == "name":
            break
        elif element.tag == "p" and element.text:
            
            """Für jeden Paragraphen der jeweiligen Äußerung schaffen wir mithilfe derselben 'SubElement'-Funktion 
            ein Element namens 'p' (zweites Argument), das dem jeweiligen 'statement'-Element untergeordnet sein soll 
            (erstes Argument). Anschließend weisen wir 'p' Textinhalt über das Attribut 'text' zu."""
            paragraph = ET.SubElement(statement, "p")
            paragraph.text = element.text.strip()           

Bevor wir dieses XML-Objekt extern speichern, überprüfen wir, ob alles geklappt hat. Hierfür lohnt es sich, Einrückungen entsprechend der Hierarchie einzufügen. Das können wir wie folgt mit einem weiteren Teil von `xml` erledigen (`minidom`):

In [23]:
from xml.dom import minidom

pretty_corpus = minidom.parseString(ET.tostring(corpus)).toprettyxml(indent="  ")

print(pretty_corpus)

<?xml version="1.0" ?>
<corpus date="10.04.2024">
  <statement speaker="Annalena Baerbock">
    <p>Einen schönen guten Tag, Frau Präsidentin! Liebe Kolleginnen und Kollegen! Als ich im Oktober zum letzten Mal hier in der Regierungsbefragung stand, war der furchtbare Angriff der Hamas auf Israel erst wenige Tage her. Seitdem, seit sechs Monaten, treiben uns Sorgen um die Geiseln und das unsägliche Leid der Menschen in Gaza um. Zwischendurch haben wir in all den Verhandlungen immer wieder nicht nur gehofft, dass wir diesen Krieg beenden können, sondern auch auf eine Chance auf einen politischen Horizont, auf ein Ende des Konflikts gehofft; aber leider sind wir dort nicht.</p>
    <p>Zugleich haben wir, habe ich persönlich erlebt, was Außenpolitik in diesen Zeiten bewirken kann, vor allen Dingen Pendeldiplomatie und Außenpolitik, die vielfach hinter verschlossenen Türen stattfindet. Sie ist aus meiner Sicht in diesen so herausfordernden Zeiten wichtiger denn je, weil unser beharrliches Ri

Schaut gut aus. 

Nun können wir das schön formatierte `pretty_corpus` wie gewohnt extern speichern:

In [24]:
with open("../3_Dateien/Output/aeusserungen_im_bundestag.xml", "w", encoding="utf-8") as write_file:
    write_file.write(pretty_corpus)

Überprüf, ob das funktioniert hat, indem Du die neu geschaffene Datei mit Sublime Text öffnest. 

Bemerkung am Rande: Für die Datenspeicherung sind die eingefügten Einrückungen natürlich unnötig. Manchmal öffnen wir XML-Dokumente aber eben auch mit grafischen Programmen wie Sublime Text. Ohne Einrückungen wäre das gesamte XML-Dokument auf eine einzigen, langen Zeile geschrieben worden, was für das menschliche Auge denkbar schwierig zu lesen ist.

Abschließend üben wir das Schreiben von XML und machen uns ein paar konzeptionelle Gedanken zu verschiedenen Speicherformaten.

*** 

✏️ **Übung 3:** Im Anwendungsfall haben wir sämtliche Strophen von Faust I in einer externen Textdatei gespeichert. Diese verfügt (idealerweise) über eine interne Struktur. In der Musterlösung wurde etwa mit Tabstopps und Zeilenumbrüchen gearbeitet, um einzelne Strophen bzw. die zugehörigen Figuren voneinander abzugrenzen. Diese Struktur kommt uns spätestens dann zu Gute, wenn wir die Daten für irgendeine Form von Auswertung mit Python wieder einlesen. 

Um unseren Anwendungsfall zu "professionalisieren", ist es nun Deine Aufgabe, den Faust I in einem XML-Dokument zu speichern, also in einem Format, dessen "Aufgabe" es ist, Daten ordentlich zu strukturieren. Kopier dazu den Code vom Extraktionsschritt des Anwendungsfalls in die folgende Zelle. Pass ihn anschließend so an, dass die Daten dynamisch in ein XML-Dokument statt in eine Textdatei geschrieben werden.

Als kleiner Bonus, der überprüft, ob bei der Speicherung alles geklappt hat, vor allem aber die Nützlichkeit strukturierter Daten aufzeigt, kannst Du Dir anschließend über die bereits gegebene Code-Zelle einfach alle Verse des berühmten Teufels aus dem Faust ausgeben lassen. Wenn nötig, pass den Dateipfad bzw. die Tags und Attribute Deiner Namensgebung an.

<details><summary>📌 Herausforderung </summary>
<br>Extrahier zusätzlich den Titel jeder Szene und speicher diese Information jeweils an der richtigen Position in der Hierarchie des zu schaffenden XML-Dokuments.
</details>
<br>

In [25]:
#In diese Zelle kannst Du den Code zur Übung schreiben.




In [26]:
#Bonus: Ausgabe aller Verse von Mephisto(pheles)
root = ET.parse("../3_Dateien/Output/faust_1.xml").getroot()

for character in root.iter("character"):
    if not character.get("name") == "Mephistopheles":
        continue
    for line in character.iter("line"):
        print(line.text)

FileNotFoundError: [Errno 2] No such file or directory: '../3_Dateien/Output/faust_1.xml'

***

✏️ **Übung 4:** XML ist wie gesagt ein beliebtes Format, um Daten zu speichern und zu teilen. Neben XML arbeiten wir beim Programmieren auch oft mit csv-Dateien (vgl. Notebooks "Input und Output" sowie "Datenanalyse"). Welchen Vorteil hat XML im Gegensatz zu csv?

<details>
<summary>💡 Tipp</summary>
<br>Analysier dazu folgenden Screenshot einer XML- bzw. csv-Datei, die fast die gleichen Daten beinhalten.
    <br><br>
    <img src="../3_Dateien/Grafiken_und_Videos/vergleich_xml_html.png">
    <br><br>
</details>

***

Wirklich gute Arbeit! 

Damit sind wir am Ende des Notebooks angelangt. 🎉

<br><br>
***
<table>
      <tr>
        <td>
            <img src="../3_Dateien/Lizenz/CC-BY-SA.png" width="400">
        </td> 
        <td>
            <p>Dieses Notebook sowie sämtliche weiteren <a href="https://github.com/yannickfrommherz/exdimed-student/tree/main">Materialien zum Programmierenlernen für Geistes- und Sozialwissenschaftler:innen</a> sind im Rahmen des Projekts <i>Experimentierraum Digitale Medienkompetenz</i> als Teil von <a href="https://tu-dresden.de/gsw/virtuos/">virTUos</a> entstanden. Erstellt wurden sie von Yannick Frommherz unter Mitarbeit von Anne Josephine Matz. Sie stehen als Open Educational Resource nach <a href="https://creativecommons.org/licenses/by-sa/4.0/">CC BY SA</a> zur freien Verfügung. Für Feedback und bei Fragen nutz bitte das <a href="https://forms.gle/VsYJgy4bZTSqKioA7">Kontaktformular</a>.
        </td>
      </tr>
</table>